In [1]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 1. Get Current Fantasy Points

In [2]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Josh Allen,BUF,QB,6,172.9
1,Nick Chubb,CLE,RB,6,119.0
2,Travis Kelce,KAN,TE,6,131.0
3,Stefon Diggs,BUF,WR,6,150.6
4,Saquon Barkley,NYG,RB,6,124.1
...,...,...,...,...,...
325,Giovanni Ricci,CAR,RB,6,10.8
326,Dalton Schultz,DAL,TE,4,15.0
335,Quintin Morris,BUF,TE,5,10.7
336,Albert Okwuegbunam,DEN,TE,5,12.0


### 2. Get Projections

##### 2a. My Projections

In [3]:
with open('../projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Preds_adp','Prob']]

##### 2b. ESPN Projections

In [4]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}

games_played = {'ARI' : 6,
 'ATL' : 6,
 'BAL' : 6,
 'BUF' : 6,
 'CAR' : 6,
 'CHI' : 6,
 'CIN' : 6,
 'CLE' : 6,
 'DAL' : 6,
 'DEN' : 6,
 'DET' : 5,
 'GNB' : 6,
 'HOU' : 5,
 'IND' : 6,
 'JAX' : 6,
 'KAN' : 6,
 'LAC' : 6,
 'LAR' : 6,
 'LVR' : 5,
 'MIA' : 6,
 'MIN' : 6,
 'NWE' : 6,
 'NOR' : 6,
 'NYG' : 6,
 'NYJ' : 6,
 'PHI' : 6,
 'PIT' : 6,
 'SEA' : 6,
 'SFO' : 6,
 'TAM' : 6,
 'TEN' : 5,
 'WAS' : 6,
 '2TM' : 6}


In [9]:
def getESPNprojs(link = '../projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()

,Player,Tm,Position,Points
0,Josh Allen,BUF,QB,374.6
1,Patrick Mahomes,KAN,QB,352.0
2,Justin Herbert,LAC,QB,328.0
3,Jalen Hurts,PHI,QB,318.5
4,Jonathan Taylor,IND,RB,317.0
...,...,...,...,...
499,Juwan Johnson,NOR,TE,4.3
500,Mike Thomas,CIN,WR,4.2
501,Dezmon Patmon,IND,WR,4.2
502,Denzel Mims,NYJ,WR,4.1


In [16]:
def mergeProjections(currPts, proj, proj_espn):
    """ 1. Merge on my projections """
    newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
    # print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
    newPts.loc[newPts['_merge']!='both', 'Preds'] = 0

    """ 2. Merge on ESPN projections"""
    newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
    # print(newerPts['espn'].value_counts())
    newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

    """ 3. Mark guys without projections"""
    newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
    newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
    
    """ 4. Prorate PPR metric so far"""
    newerPts['GP'] = pd.to_numeric(newerPts['Tm'].replace(games_played))
    newerPts['PPR_adj'] = newerPts['PPR'] * newerPts['GP']

    newerPts.rename(columns = {'Points' : 'Points_ESPN'}, inplace = True)
    return newerPts
newerPts = mergeProjections(currPts, proj, proj_espn)
# newerPts.to_excel('test.xlsx')

In [19]:
newerPts['Points_ESPN'].isnull().sum()

0

In [21]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points_ESPN'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Preds_adp'].corr(newerPts['PPR_adj'])**2)

newerPts['PPR_adj']

0.5598453726051544
0.5325826245051606
0.46799602989981565


0      1037.4
1       714.0
2       786.0
3       903.6
4       744.6
        ...  
307      64.8
308      90.0
309      64.2
310      72.0
311      59.0
Name: PPR_adj, Length: 312, dtype: float64